# 🔍 RAG na Prática: Transformando Documentos em Conversas Inteligentes!

**Módulo 8 - LangChain v0.2**

Fala pessoal! 🚀 Chegou a hora de juntar todas as peças do quebra-cabeças que vimos até agora! Lembram dos **Document Loaders**, **Text Splitters**, **Vector Stores** e **Embeddings**? Pois é, agora vamos usar TUDO isso para criar um sistema RAG completo!

Pensa assim: é como se você fosse montar um assistente pessoal que leu TODOS os seus documentos e pode responder qualquer pergunta sobre eles. Lindão, né?

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-08_img_01.png)

## 🤔 Tá, mas o que é RAG mesmo?

**RAG** significa **Retrieval-Augmented Generation** (Geração Aumentada por Recuperação). Parece complicado, mas é bem simples:

### A Analogia da Biblioteca Inteligente 📚

Imagina que você tem uma biblioteca gigantesca e um bibliotecário super inteligente:

1. **Você faz uma pergunta** 🙋‍♂️
2. **O bibliotecário procura** nos livros certos 📖
3. **Ele encontra informações relevantes** ✨
4. **Ele usa essas informações para te dar uma resposta completa** 💡

É exatamente isso que o RAG faz! A diferença é que:
- A **biblioteca** são seus documentos
- O **bibliotecário** é o LLM (Gemini, GPT, etc.)
- A **busca** é feita por similaridade semântica

### Por que RAG é Revolucionário? 🌟

- **Conhecimento Atualizado**: Não precisa retreinar o modelo
- **Fonte Confiável**: Baseado nos SEUS documentos
- **Transparência**: Você sabe de onde veio a resposta
- **Eficiência**: Muito mais barato que fine-tuning

**Dica do Pedro**: RAG é como dar superpoderes para o LLM. Ele continua sendo inteligente, mas agora tem acesso aos seus dados específicos!

## 🏗️ Arquitetura do RAG: Como Funciona por Dentro?

Vamos entender o fluxo completo do RAG:

```mermaid
graph TD
    A[📄 Documentos] --> B[🔪 Text Splitter]
    B --> C[🧠 Embeddings]
    C --> D[💾 Vector Store]
    E[❓ Pergunta do Usuário] --> F[🔍 Retriever]
    D --> F
    F --> G[📝 Contexto Relevante]
    G --> H[🤖 LLM + Prompt]
    E --> H
    H --> I[✨ Resposta Final]
```

### Os Dois Momentos do RAG:

#### 1️⃣ **Momento da Indexação** (Fazemos UMA vez)
- Carregamos documentos
- Dividimos em chunks
- Criamos embeddings
- Guardamos no vector store

#### 2️⃣ **Momento da Consulta** (Fazemos SEMPRE)
- Recebemos pergunta
- Buscamos chunks similares
- Montamos prompt com contexto
- LLM gera resposta

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-08_img_02.png)

In [ ]:
# Bora começar! Primeiro, vamos instalar e importar tudo que precisamos
# Esses são os imports essenciais para RAG com LangChain v0.2

!pip install -q langchain langchain-google-genai langchain-community faiss-cpu pypdf

import os
from google.colab import userdata

# Configurar API key do Google
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Imports do LangChain - Tudo que vimos nos módulos anteriores!
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_community.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

print("🚀 Tudo importado! Bora implementar RAG!")

In [ ]:
# Vamos criar nossos componentes básicos - Lembram deles?

# 1. LLM - Nosso gemini-2.0-flash de sempre
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.1,  # Baixa criatividade para respostas mais precisas
    max_tokens=1000
)

# 2. Embeddings - Para transformar texto em vetores
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

# 3. Text Splitter - Para dividir documentos em chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Tamanho de cada pedaço
    chunk_overlap=200,    # Sobreposição entre pedaços
    separators=["\n\n", "\n", ".", " "]  # Como dividir
)

print("✅ Componentes básicos criados!")
print(f"📱 LLM: {llm.model_name}")
print(f"🧠 Embeddings: {embeddings.model}")
print(f"✂️ Chunk size: {text_splitter._chunk_size}")

## 📚 Fase 1: Preparando os Documentos (Indexação)

Agora vamos criar alguns documentos de exemplo para nosso RAG. Na vida real, vocês vão usar PDFs, páginas web, bancos de dados, etc.

**Dica do Pedro**: Esta é a parte mais importante! A qualidade dos seus documentos determina a qualidade das respostas. Lixo entra, lixo sai! 🗑️➡️🗑️

In [ ]:
# Vamos criar documentos de exemplo sobre tecnologia brasileira
# Na prática, vocês carregariam PDFs, sites, etc.

documentos_exemplo = [
    {
        "conteudo": """
        A Embraer é uma das maiores fabricantes de aeronaves do mundo, fundada em 1969 em São José dos Campos, SP.
        A empresa produz jatos regionais e executivos, além de aeronaves militares.
        Seus principais produtos incluem a família E-Jets e os jatos executivos Phenom e Praetor.
        A Embraer emprega mais de 18.000 pessoas e exporta para mais de 100 países.
        """,
        "metadata": {"fonte": "embraer_info.txt", "categoria": "aviacao"}
    },
    {
        "conteudo": """
        O Nubank foi fundado em 2013 por David Vélez, Cristina Junqueira e Edward Wible.
        É considerado o maior banco digital da América Latina, com mais de 80 milhões de clientes.
        A empresa revolucionou o mercado financeiro brasileiro com seu cartão de crédito sem anuidade.
        Em 2021, o Nubank abriu capital na NYSE, sendo uma das maiores IPOs da história brasileira.
        """,
        "metadata": {"fonte": "nubank_historia.txt", "categoria": "fintech"}
    },
    {
        "conteudo": """
        A inteligência artificial no Brasil tem crescido exponencialmente nos últimos anos.
        Empresas como C6 Bank, Itaú e Bradesco investem pesado em IA para melhorar serviços.
        O Brasil possui mais de 400 startups de IA, com foco em fintechs, agrotechs e healthtechs.
        Universidades como USP, UNICAMP e PUC lideram pesquisas em machine learning e deep learning.
        """,
        "metadata": {"fonte": "ia_brasil.txt", "categoria": "tecnologia"}
    }
]

print(f"📄 Criamos {len(documentos_exemplo)} documentos de exemplo")
print("📋 Categorias:", [doc["metadata"]["categoria"] for doc in documentos_exemplo])

In [ ]:
# Agora vamos processar os documentos - Exatamente como vimos nos módulos anteriores!
from langchain.schema import Document

# 1. Converter para objetos Document do LangChain
documents = []
for doc in documentos_exemplo:
    documents.append(
        Document(
            page_content=doc["conteudo"].strip(),
            metadata=doc["metadata"]
        )
    )

print(f"📄 {len(documents)} documentos carregados")

# 2. Dividir em chunks menores
chunks = text_splitter.split_documents(documents)

print(f"✂️ Divididos em {len(chunks)} chunks")

# Vamos ver como ficaram os chunks
for i, chunk in enumerate(chunks):
    print(f"\n📝 Chunk {i+1}:")
    print(f"   Tamanho: {len(chunk.page_content)} caracteres")
    print(f"   Fonte: {chunk.metadata['fonte']}")
    print(f"   Prévia: {chunk.page_content[:100]}...")

In [ ]:
# 3. Criar embeddings e guardar no Vector Store
# Aqui é onde a mágica acontece! 🪄

print("🔄 Criando embeddings e construindo vector store...")
print("(Isso pode levar alguns segundos...)")

# Criar o FAISS vector store com nossos chunks
vector_store = FAISS.from_documents(chunks, embeddings)

print("✅ Vector store criado com sucesso!")
print(f"📊 Armazenados {vector_store.index.ntotal} vetores")
print(f"🔢 Dimensão dos vetores: {vector_store.index.d}")

# Criar o retriever - Quem vai buscar documentos similares
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Buscar os 3 chunks mais similares
)

print("🔍 Retriever configurado para buscar os 3 documentos mais similares")

## 🧪 Testando o Retriever

Antes de montar a chain completa, vamos testar se nosso retriever está funcionando direitinho. É como testar se o bibliotecário consegue encontrar os livros certos!

**Dica do Pedro**: Sempre teste o retriever separadamente! Se ele não trouxer documentos relevantes, o RAG inteiro vai dar ruim! 🎯

In [ ]:
# Vamos testar algumas perguntas para ver se o retriever funciona
perguntas_teste = [
    "Quem fundou o Nubank?",
    "O que a Embraer produz?",
    "Como está a IA no Brasil?"
]

for pergunta in perguntas_teste:
    print(f"\n❓ Pergunta: {pergunta}")
    print("=" * 50)
    
    # Buscar documentos relevantes
    docs_relevantes = retriever.get_relevant_documents(pergunta)
    
    for i, doc in enumerate(docs_relevantes):
        print(f"\n📄 Documento {i+1} (Fonte: {doc.metadata['fonte']})")
        print(f"🏷️ Categoria: {doc.metadata['categoria']}")
        print(f"📝 Conteúdo: {doc.page_content[:200]}...")
        
print("\n🎉 Retriever funcionando perfeitamente!")

## 🔗 Montando a Chain RAG Completa

Agora vem a parte mais legal! Vamos juntar tudo em uma chain que:
1. Recebe uma pergunta
2. Busca documentos relevantes
3. Monta um prompt com contexto
4. Gera uma resposta baseada nos documentos

### A Matemática por trás do RAG:

$$RAG(query) = LLM(prompt + context + query)$$

Onde:
- $context = Retriever(embed(query), VectorStore)$
- $Retriever$ usa similaridade coseno: $sim(a,b) = \frac{a \cdot b}{||a|| \cdot ||b||}$

**Dica do Pedro**: O segredo está no prompt! Ele precisa instruir o LLM a usar APENAS o contexto fornecido. Senão ele vai "alucinar" informações! 🤖💭

In [ ]:
# Vamos criar o prompt template para RAG
# Este é o coração do sistema!

system_prompt = """
Você é um assistente especializado em tecnologia e empresas brasileiras.
Use APENAS as informações fornecidas no contexto abaixo para responder às perguntas.

REGRAS IMPORTANTES:
1. Se a informação não estiver no contexto, diga "Não tenho essa informação nos documentos fornecidos"
2. Seja preciso e cite informações específicas do contexto
3. Responda em português brasileiro
4. Seja conversacional mas profissional

Contexto:
{context}

Pergunta: {input}

Resposta:
"""

# Criar o prompt template
prompt = ChatPromptTemplate.from_template(system_prompt)

print("✅ Prompt template criado!")
print("🎯 O prompt vai instruir o LLM a usar APENAS o contexto fornecido")

In [ ]:
# Agora vamos montar as chains!
# No LangChain v0.2, usamos essas funções específicas

# 1. Chain para processar documentos e gerar resposta
document_chain = create_stuff_documents_chain(llm, prompt)

# 2. Chain completa de RAG (retriever + document chain)
rag_chain = create_retrieval_chain(retriever, document_chain)

print("🔗 RAG Chain montada com sucesso!")
print("🚀 Sistema RAG completo pronto para uso!")

# Vamos ver a estrutura da chain
print("\n📋 Componentes da RAG Chain:")
print(f"   🔍 Retriever: {type(retriever).__name__}")
print(f"   🤖 LLM: {llm.model_name}")
print(f"   📝 Prompt: Personalizado para RAG")
print(f"   💾 Vector Store: FAISS com {vector_store.index.ntotal} documentos")

## 🎯 Hora do Show: Testando o RAG!

Agora vem a parte mais divertida! Vamos fazer perguntas para nosso sistema RAG e ver como ele se comporta. 

Reparem que as respostas vão incluir:
- **answer**: A resposta gerada pelo LLM
- **context**: Os documentos que foram usados como base

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-08_img_03.png)

In [ ]:
# Função para testar o RAG de forma organizada
def testar_rag(pergunta):
    print(f"\n❓ PERGUNTA: {pergunta}")
    print("=" * 60)
    
    # Fazer a pergunta para o RAG
    resultado = rag_chain.invoke({"input": pergunta})
    
    # Mostrar a resposta
    print(f"\n🤖 RESPOSTA:")
    print(resultado["answer"])
    
    # Mostrar os documentos que foram usados
    print(f"\n📚 DOCUMENTOS CONSULTADOS:")
    for i, doc in enumerate(resultado["context"]):
        print(f"   {i+1}. {doc.metadata['fonte']} ({doc.metadata['categoria']})")
    
    return resultado

# Vamos testar!
perguntas = [
    "Quem são os fundadores do Nubank?",
    "Onde fica a sede da Embraer?", 
    "Quantos clientes o Nubank tem?"
]

resultados = []
for pergunta in perguntas:
    resultado = testar_rag(pergunta)
    resultados.append(resultado)
    print("\n" + "─" * 80)

In [ ]:
# Vamos testar uma pergunta que NÃO está nos documentos
# Para ver se o sistema vai "alucinar" ou ser honesto

pergunta_inexistente = "Qual é o faturamento anual da Magazine Luiza?"
resultado_teste = testar_rag(pergunta_inexistente)

print("\n🧪 TESTE DE INTEGRIDADE:")
print("Perguntamos sobre algo que não está nos documentos.")
print("Um bom sistema RAG deve admitir que não tem a informação!")

## 📊 Visualizando o Desempenho do RAG

Vamos criar algumas visualizações para entender melhor como nosso RAG está funcionando!

**Dica do Pedro**: Sempre meça o desempenho do seu RAG! Não adianta só "achar" que está bom. Dados não mentem! 📈

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# Análise dos documentos recuperados
fontes_usadas = []
categorias_usadas = []

for resultado in resultados:
    for doc in resultado["context"]:
        fontes_usadas.append(doc.metadata["fonte"])
        categorias_usadas.append(doc.metadata["categoria"])

# Criar visualizações
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Fontes mais consultadas
fontes_count = Counter(fontes_usadas)
ax1.bar(range(len(fontes_count)), list(fontes_count.values()), color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax1.set_xticks(range(len(fontes_count)))
ax1.set_xticklabels([f.replace('_', '\n') for f in fontes_count.keys()], rotation=45)
ax1.set_title('📄 Fontes Mais Consultadas pelo RAG', fontsize=14, fontweight='bold')
ax1.set_ylabel('Número de Consultas')

# Gráfico 2: Categorias mais relevantes
cat_count = Counter(categorias_usadas)
colors = ['#ff9999', '#66b3ff', '#99ff99']
wedges, texts, autotexts = ax2.pie(cat_count.values(), labels=cat_count.keys(), autopct='%1.1f%%', colors=colors)
ax2.set_title('🏷️ Distribuição por Categoria', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("📊 Análise do RAG:")
print(f"   • Total de documentos consultados: {len(fontes_usadas)}")
print(f"   • Fonte mais usada: {fontes_count.most_common(1)[0][0]}")
print(f"   • Categoria mais relevante: {cat_count.most_common(1)[0][0]}")

In [ ]:
# Vamos simular scores de similaridade para visualizar
# Na prática, vocês podem extrair isso do FAISS

import random
random.seed(42)

# Simular scores de similaridade para as consultas
consultas = ["Nubank fundadores", "Embraer sede", "Clientes Nubank"]
scores = [
    [0.89, 0.76, 0.52],  # Scores para consulta 1
    [0.91, 0.68, 0.45],  # Scores para consulta 2
    [0.87, 0.73, 0.49]   # Scores para consulta 3
]

# Criar gráfico de similaridade
fig, ax = plt.subplots(figsize=(12, 8))

x = np.arange(len(consultas))
width = 0.25

# Criar barras para cada documento recuperado
for i in range(3):
    scores_doc = [score[i] for score in scores]
    ax.bar(x + i*width, scores_doc, width, label=f'Doc {i+1}', alpha=0.8)

ax.set_xlabel('Consultas RAG', fontsize=12)
ax.set_ylabel('Score de Similaridade', fontsize=12)
ax.set_title('🎯 Scores de Similaridade por Consulta', fontsize=14, fontweight='bold')
ax.set_xticks(x + width)
ax.set_xticklabels(consultas)
ax.legend()
ax.grid(True, alpha=0.3)

# Adicionar linha de corte para "boa similaridade"
ax.axhline(y=0.7, color='red', linestyle='--', alpha=0.7, label='Threshold (0.7)')
ax.legend()

plt.tight_layout()
plt.show()

print("🎯 Análise de Similaridade:")
print("   • Scores acima de 0.7 são considerados bons")
print("   • RAG funcionando bem - docs relevantes têm alta similaridade")
print("   • Docs com baixa similaridade servem como contexto adicional")

## 🚀 RAG Avançado: Melhorando o Sistema

Nosso RAG básico já funciona, mas podemos melhorar muito! Vamos implementar algumas técnicas avançadas:

### 1. **Filtros por Metadata** 🔍
### 2. **Reranking de Resultados** 📊
### 3. **Contexto Mais Rico** 📚

```mermaid
graph TD
    A[❓ Query] --> B[🔍 Retrieval]
    B --> C[🏷️ Metadata Filter]
    C --> D[📊 Reranking]
    D --> E[📚 Context Enhancement]
    E --> F[🤖 LLM Generation]
    F --> G[✨ Enhanced Answer]
```

**Dica do Pedro**: RAG é como tempero de comida - sempre dá para melhorar! A versão básica mata a fome, mas as técnicas avançadas fazem o prato ficar gourmet! 👨‍🍳

In [ ]:
# RAG com filtro por categoria
def rag_com_filtro(pergunta, categoria=None, k=3):
    """
    RAG que pode filtrar por categoria específica
    """
    print(f"\n🔍 Buscando: '{pergunta}'")
    if categoria:
        print(f"🏷️ Filtrando por categoria: {categoria}")
    
    # Se temos filtro de categoria, vamos buscar manualmente
    if categoria:
        # Buscar todos os documentos primeiro
        todos_docs = retriever.get_relevant_documents(pergunta)
        
        # Filtrar por categoria
        docs_filtrados = [
            doc for doc in todos_docs 
            if doc.metadata.get('categoria') == categoria
        ][:k]
        
        print(f"📄 Encontrados {len(docs_filtrados)} documentos na categoria '{categoria}'")
        
        if not docs_filtrados:
            return {"answer": f"Não encontrei documentos na categoria '{categoria}' para responder essa pergunta.", "context": []}
        
        # Montar contexto manualmente
        contexto = "\n\n".join([doc.page_content for doc in docs_filtrados])
        
        # Gerar resposta
        prompt_formatado = prompt.format(context=contexto, input=pergunta)
        resposta = llm.invoke(prompt_formatado)
        
        return {"answer": resposta.content, "context": docs_filtrados}
    else:
        # Usar RAG normal
        return rag_chain.invoke({"input": pergunta})

# Testar RAG com filtro
resultado_normal = rag_com_filtro("Conte sobre empresas brasileiras de tecnologia")
print("\n🤖 RESPOSTA (SEM FILTRO):")
print(resultado_normal["answer"])

resultado_fintech = rag_com_filtro("Conte sobre empresas brasileiras de tecnologia", categoria="fintech")
print("\n🤖 RESPOSTA (FILTRO: FINTECH):")
print(resultado_fintech["answer"])

In [ ]:
# RAG com contexto enriquecido - Incluindo metadados na resposta
def rag_enriquecido(pergunta):
    """
    Versão do RAG que inclui informações sobre as fontes
    """
    resultado = rag_chain.invoke({"input": pergunta})
    
    # Enriquecer a resposta com informações das fontes
    fontes = []
    for doc in resultado["context"]:
        fonte_info = f"📄 {doc.metadata['fonte']} (categoria: {doc.metadata['categoria']})"
        if fonte_info not in fontes:
            fontes.append(fonte_info)
    
    resposta_enriquecida = resultado["answer"]
    resposta_enriquecida += "\n\n📚 **Fontes consultadas:**\n"
    resposta_enriquecida += "\n".join(fontes)
    
    return {
        "answer": resposta_enriquecida,
        "context": resultado["context"],
        "num_fontes": len(fontes)
    }

# Testar RAG enriquecido
pergunta_teste = "Qual empresa brasileira é líder em aviação?"
resultado_enriquecido = rag_enriquecido(pergunta_teste)

print(f"❓ PERGUNTA: {pergunta_teste}")
print("=" * 60)
print(resultado_enriquecido["answer"])
print(f"\n📊 Consultou {resultado_enriquecido['num_fontes']} fontes diferentes")

## 🏋️‍♂️ Exercício Prático: Construa Seu Próprio RAG!

Agora é sua vez! Vamos criar um desafio prático para vocês aplicarem tudo que aprenderam.

### 🎯 **DESAFIO 1**: Expandir a Base de Conhecimento

Adicione mais documentos ao nosso sistema RAG sobre tecnologia brasileira. Pode ser sobre:
- Outras empresas (Mercado Livre, iFood, 99, etc.)
- Universidades e pesquisa
- Startups famosas
- Políticas de tecnologia

**Requisitos:**
1. Pelo menos 3 documentos novos
2. Categorias diferentes
3. Testar com perguntas específicas
4. Medir o desempenho

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-08_img_04.png)

In [ ]:
# EXERCÍCIO 1: Expandir a base de conhecimento
# TODO: Adicione seus documentos aqui!

# DICA: Use este formato
novos_documentos = [
    {
        "conteudo": """
        # Seu conteúdo aqui!
        # Ex: Informações sobre o iFood, Mercado Livre, etc.
        """,
        "metadata": {"fonte": "seu_arquivo.txt", "categoria": "sua_categoria"}
    },
    # Adicione mais documentos...
]

# Escreva seu código aqui para:
# 1. Converter para Documents
# 2. Fazer split dos textos
# 3. Adicionar ao vector store existente
# 4. Testar com novas perguntas

print("💡 DICA: Use vector_store.add_documents() para adicionar novos documentos!")
print("🔄 Não esqueça de testar o retriever após adicionar os documentos!")

# SEU CÓDIGO AQUI:


## 🏋️‍♂️ Exercício 2: RAG com Diferentes Tipos de Busca

### 🎯 **DESAFIO 2**: Implementar Diferentes Estratégias de Retrieval

O FAISS oferece diferentes tipos de busca. Vamos experimentar:
- **Similarity**: Busca por similaridade (que já usamos)
- **MMR (Maximal Marginal Relevance)**: Busca diversificada
- **Similarity with threshold**: Busca com limite de similaridade

**Sua missão**: Implementar e comparar essas estratégias!

**Dica do Pedro**: Cada estratégia tem seus prós e contras. MMR é ótimo quando você quer diversidade, threshold quando você quer qualidade garantida! 🎯

In [ ]:
# EXERCÍCIO 2: Diferentes estratégias de retrieval

def comparar_estrategias_retrieval(pergunta):
    """
    Compara diferentes estratégias de busca no mesmo vector store
    """
    print(f"🔍 Testando estratégias para: '{pergunta}'")
    print("=" * 60)
    
    # TODO: Implementar diferentes retrievers
    
    # 1. Similarity (já temos)
    retriever_similarity = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    
    # 2. MMR - Máxima Relevância Marginal
    # TODO: Criar retriever MMR
    # DICA: search_type="mmr", search_kwargs={"k": 3, "fetch_k": 6}
    
    # 3. Similarity com threshold
    # TODO: Criar retriever com threshold
    # DICA: search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7}
    
    # Testar cada um e comparar resultados
    
    return None

# Teste suas implementações
pergunta_teste = "Como está o mercado de IA no Brasil?"
# comparar_estrategias_retrieval(pergunta_teste)

print("🚧 Implemente os diferentes retrievers no código acima!")
print("📊 Compare os resultados e veja qual funciona melhor para cada tipo de pergunta")

## 🔄 Conectando com os Próximos Módulos

Lindão! Vocês acabaram de dominar RAG! 🎉 Mas a jornada não para aqui...

### 🔮 **O que vem pela frente:**

#### **Módulo 9 - Agents e Tools** 🤖
- Vamos dar "superpoderes" para o RAG
- Agents que decidem QUANDO usar RAG
- Tools que permitem buscar na internet, fazer cálculos, etc.
- RAG + Agents = 🤯

#### **Módulos 10-11 - Projetos Finais** 🚀
- Sistema RAG completo para empresa real
- RAG + Streamlit = aplicação web
- Deploy na AWS (que vai ser lindo!)

### 🧠 **Conceitos que vamos expandir:**

```mermaid
graph TD
    A[RAG Atual] --> B[RAG + Agents]
    B --> C[RAG + Tools]
    C --> D[RAG + Web Interface]
    D --> E[RAG em Produção]
    E --> F[RAG + Monitoring]
```

**Dica do Pedro**: RAG é a base! Tudo que vem depois vai usar os conceitos que vocês aprenderam aqui. É como aprender a dirigir - depois vocês vão dirigir carros mais avançados, mas o básico é sempre o mesmo! 🚗➡️🏎️

In [ ]:
# Visualização: Evolução do RAG ao longo do curso
import matplotlib.pyplot as plt
import numpy as np

# Dados da evolução
modulos = ['M6\nDocs', 'M7\nVectors', 'M8\nRAG', 'M9\nAgents', 'M10-11\nProjetos', 'M12\nDeploy']
complexidade = [2, 4, 6, 8, 9, 10]
utilidade = [1, 3, 7, 9, 10, 10]

fig, ax = plt.subplots(figsize=(12, 8))

# Plotar as curvas
x = np.arange(len(modulos))
ax.plot(x, complexidade, 'o-', linewidth=3, markersize=8, label='🧠 Complexidade', color='#ff6b6b')
ax.plot(x, utilidade, 'o-', linewidth=3, markersize=8, label='⚡ Utilidade Prática', color='#4ecdc4')

# Destacar o módulo atual
ax.axvline(x=2, color='gold', linestyle='--', alpha=0.7, linewidth=2)
ax.text(2, 8.5, '← VOCÊ ESTÁ AQUI!', fontsize=12, fontweight='bold', 
        ha='center', bbox=dict(boxstyle='round,pad=0.3', facecolor='gold', alpha=0.7))

# Customizar o gráfico
ax.set_xlabel('Módulos do Curso', fontsize=12, fontweight='bold')
ax.set_ylabel('Nível (1-10)', fontsize=12, fontweight='bold')
ax.set_title('🚀 Evolução do Conhecimento em RAG/LangChain', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(modulos)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 11)

# Adicionar anotações
ax.annotate('Fundamentos', xy=(1, 3), xytext=(0.5, 5),
            arrowprops=dict(arrowstyle='->', color='gray', alpha=0.7),
            fontsize=10, ha='center')

ax.annotate('RAG Completo!', xy=(2, 7), xytext=(2, 5),
            arrowprops=dict(arrowstyle='->', color='green', alpha=0.7),
            fontsize=10, ha='center', color='green', fontweight='bold')

ax.annotate('Produção', xy=(5, 10), xytext=(4.5, 8.5),
            arrowprops=dict(arrowstyle='->', color='blue', alpha=0.7),
            fontsize=10, ha='center')

plt.tight_layout()
plt.show()

print("📈 Análise da Jornada:")
print(f"   • Complexidade atual: {complexidade[2]}/10 - Moderada")
print(f"   • Utilidade atual: {utilidade[2]}/10 - Alta!")
print("   • Próximos passos: Agents (complexidade +2, utilidade +2)")
print("   • Objetivo: Produção (complexidade +4, utilidade +3)")

## 📋 Resumo do Módulo: O que Aprendemos?

Caramba, que módulo intenso! 🤯 Vamos recapitular tudo que rolou:

### ✅ **Conceitos Dominados:**

1. **RAG Architecture** 🏗️
   - Retrieval-Augmented Generation
   - Fase de indexação vs. fase de consulta
   - Como juntar retrieval + generation

2. **Implementação Prática** 💻
   - `create_retrieval_chain()` e `create_stuff_documents_chain()`
   - Integration com componentes dos módulos anteriores
   - Prompt engineering para RAG

3. **Técnicas Avançadas** 🚀
   - Filtros por metadata
   - Diferentes estratégias de retrieval
   - Context enrichment
   - Performance monitoring

### 📊 **Fórmulas e Conceitos Técnicos:**

$$RAG(query) = LLM(prompt + context + query)$$
$$context = Retriever(embed(query), VectorStore)$$
$$similarity(a,b) = \frac{a \cdot b}{||a|| \cdot ||b||}$$

### 🎯 **Aplicações Reais:**
- Chatbots corporativos
- Sistemas de suporte técnico
- Assistentes de pesquisa
- Q&A sobre documentação

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-08_img_05.png)

**Dica do Pedro Final**: RAG mudou o jogo da IA! Vocês agora sabem como fazer LLMs conversarem com qualquer base de dados. Isso vale ouro no mercado! 💰✨

In [ ]:
# Parabéns! Vamos celebrar com algumas estatísticas finais
import datetime

print("🎉 PARABÉNS! Você completou o Módulo 8 - RAG Implementation!")
print("=" * 60)

# Estatísticas do que foi construído
stats = {
    "Documentos processados": len(documents),
    "Chunks criados": len(chunks),
    "Embeddings gerados": vector_store.index.ntotal if 'vector_store' in locals() else 0,
    "Dimensão dos vetores": vector_store.index.d if 'vector_store' in locals() else 0,
    "Consultas RAG testadas": len(resultados) if 'resultados' in locals() else 0,
    "Chains criadas": 2,  # document_chain + rag_chain
    "Componentes integrados": 4  # LLM, Embeddings, VectorStore, Retriever
}

print("📊 ESTATÍSTICAS DA SUA IMPLEMENTAÇÃO RAG:")
for key, value in stats.items():
    print(f"   • {key}: {value}")

print("\n🚀 PRÓXIMOS PASSOS:")
print("   • Módulo 9: Agents e Tools - RAG com superpoderes!")
print("   • Módulo 10-11: Projetos finais - RAG em aplicações reais")
print("   • Módulo 12: Deploy - RAG em produção com Streamlit")

print("\n💡 DICAS PARA CONTINUAR ESTUDANDO:")
print("   • Teste com seus próprios documentos")
print("   • Experimente diferentes embeddings")
print("   • Implemente os exercícios propostos")
print("   • Leia sobre RAG avançado (GraphRAG, MultiVector RAG, etc.)")

print(f"\n📅 Concluído em: {datetime.datetime.now().strftime('%d/%m/%Y às %H:%M')}")
print("\n🏆 VOCÊ AGORA É UM RAG MASTER! Liiindo! 🇧🇷🚀")